❗이 튜토리얼은 Ultralytics의 [YOLO11 tutorial colab notebook](https://colab.research.google.com/github/ultralytics/ultralytics/blob/main/examples/tutorial.ipynb)을 참고하여 작성되었습니다.

# Setup

**라이브러리 설치**

우선 YOLO 11을 사용하기 위해 Ultralytics의 라이브러리를 설치합니다.

In [ ]:
# 1분 이상 소요
%pip install ultralytics

**설치 확인**

다음 문구가 출력되면 성공입니다.

```
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.8/107.7 GB disk)
```

In [ ]:
import ultralytics
ultralytics.checks()

# Ultralytics 사용법




Ultralytics 라이브러리는 **Python**과 **CLI(Command line)** 두 가지 방법으로 이용할 수 있습니다.


**Python 사용 예**

```
from ultralytics import YOLO

# 1) 모델 로드
model = YOLO('yolov8n.pt')  

# 2) 학습
model.train(
    data='data.yaml',     # 데이터 YAML 파일
    epochs=50,            # 반복 횟수
    imgsz=640,            # 입력 이미지 크기
    batch=16,             # 배치 크기
    lr0=1e-3,             # 초기 학습률
    project='runs/train', # 결과 저장 폴더
    name='exp1'           # 실험 이름
)

# 3) 추론
model('img_path')

# 4) 평가
model.val()

```

장점: Python 코드 안에서 모든 설정을 제어할 수 있으며 실험 파이프라인에 유연하게 통합 가능

참고([링크](https://docs.ultralytics.com/ko/usage/python/))

**CLI(Command-Line) 사용 예**

터미널 혹은 Colab 셀에서 바로 실행합니다.
```
yolo TASK MODE ARGS
```


* TASK (선택 사항)은 다음 중 하나입니다. [detect, segment, classify, pose, obb]. 명시적으로 전달되지 않은 경우 YOLO TASK 를 모델 유형에서 선택합니다.
* MODE (필수)는 다음 중 하나입니다. [train, val, predict, export, track, benchmark]
* ARGS (선택 사항) 사용자 지정 arg=value 같은 imgsz=320 기본값을 재정의할 수 있습니다. 사용 가능한 전체 목록은 ARGS를 참조하세요. 구성 페이지와 defaults.yaml.

학습
```
yolo detect train data=data.yaml model=yolov8n.pt epochs=50 imgsz=640 batch=16 lr0=1e-3 project=runs/train name=exp1
```

예측
```
yolo predict model=yolo11n.pt source='https://youtu.be/LNwODJXcvt4' imgsz=320
```
* `source`는 예측 대상으로 이미지(한 장), 디렉토리, 링크, 비디오 등 다양한 유형이 될 수 있습니다.

평가
```
yolo val model=yolo11n.pt data=data.yaml batch=1 imgsz=640
```

장점: 명령 한 줄로 실행 — 사용이 간단하며 빠르게 설정하고 결과를 확인하기에 편리

단점: 복잡한 로직이나 세세한 설정과 실험 관리에 제약이

참고([링크](https://docs.ultralytics.com/ko/usage/cli/))

이 노트북에서는 CLI 방식으로 실험해보고, 마지막에 Python API를 활용해보겠습니다.

# 데이터셋 업로드

`데이터셋_구축.ipynb`에서 구축한 데이터셋을 업로드합니다.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
!ls # 출력: PigDataset.zip

In [ ]:
!unzip -qq PigDataset.zip

In [ ]:
!ls PigDataset # pig.yaml  train  val

# 학습

Ultralytics에서 제공하는 사전 학습 모델을 이용합니다. (참고: https://docs.ultralytics.com/ko/models/yolo11/)

모델과 간단한 하이퍼파라미터를 선택해 학습니다.


In [ ]:
# 텐서보드 설정 - 텐서보드는 모델의 학습 과정(loss curve 등)을 모니터링할 수 있는 도구입니다.
!yolo settings tensorboard=True
%load_ext tensorboard

학습

* Ultralytics의 YOLOv5 이상 버전부터는 학습 파이프라인에 Mosaic 증강이 기본으로 포함되어 있어, 별도로 증강을 설정할 필요가 없습니다.

* Mosaic 증강에 대한 자세한 내용은 [링크](https://docs.ultralytics.com/guides/yolo-data-augmentation/)를 참고하세요.
* `patience` 옵션을 추가해 일정 구간 학습이 이루어지지 않으면 **early stopping**을 합니다.

In [ ]:
!yolo train model=yolo11s.pt data=PigDataset/pig.yaml epochs=100 imgsz=640 patience=20

텐서보드로 학습 과정 및 결과를 확인합니다.

* `train` 탭의  loss 곡선과 `val` 탭의 loss 곡선을 비교하세요.
* `metrics` 탭의 AP 성능 변화를 확인하세요.

학습이 잘 되었나요?

In [ ]:
%tensorboard --logdir .

실험 결과는 별도로 설정하지 않았다면 `runs/detect/` 디렉토리에 저장됩니다.

다시 학습을 하면 'train'부터 순서대로 'train2', 'train3', .. 순으로 자동으로 번호가 지정됩니다.

이후의 코드에서는 반드시 실험 모델 경로를 확인하고 코드를 수정해주세요.

`!ls runs/detect` 명령어 혹은 파일 탭(왼쪽 메뉴 맨 아래)에서 저장된 train 폴더 목록을 확인하는 것을 추천합니다.

In [ ]:
!ls runs/detect

YOLO의 실험은 다양한 분석 결과를 함께 저장해줍니다.

왼쪽  탭의 맨 아래쪽 파일 탭을 눌러 저장된 실험 결과를 확인하세요.

설정 정보, 학습 결과, 학습 곡선, confusion matrix, 배치 이미지 등을 확인할 수 있습니다.

학습한 모델은 `weights` 디렉토리에 저장되어 있습니다.

실험결과를 모두 저장하고 싶다면 `runs` 디렉토리를 압축한 후, 다운로드 합니다. (*데이터셋_구축.ipynb* 참고)

***세션이 종료되면 실험 결과는 사라지니, 필요하다면 반드시 다운로드하세요***

In [ ]:
# 실험 디렉토리 확인
!ls runs/detect/train

In [ ]:
# 학습 모델 확인
!ls runs/detect/train/weights

# 추론

학습한 모델을 이용해 예측합니다.

검증 이미지를 모두 예측해보겠습니다.

```
!yolo predict model=runs/detect/train/weights/best.pt source=PigDataset/val/images
```


`weights`에저장된 `best.pt`와 `last.pt` 중, `best.pt` 사용을 추천합니다.

예: `runs/detect/train/weights/best.pt`

*세션에서 학습을 여러 번 진행했을 경우 반드시 실험 디렉토리를 확인하세요*

In [ ]:
!yolo predict model=runs/detect/train/weights/best.pt source=PigDataset/val/images

위 코드 실행 시 이미지 별 검출 객체 수, 실행 시간, 예측 결과 저장 경로 등을 확인할 수 있습니다.

* `Results saved to runs/detect/predict`: 저장 경로를 알려줍니다.

In [ ]:
!ls runs/detect

In [ ]:
!ls runs/detect/predict

예측 결과를 압축 후 다운로드 하여 로컬에서 확인해봅시다.

In [ ]:
!zip -r predict.zip runs/detect/predict

In [ ]:
files.download('predict.zip')

# 평가

In [ ]:
!yolo val model=runs/detect/train/weights/best.pt data=PigDataset/pig.yaml imgsz=640 batch=8 conf=0.3 iou=0.6

# Python

커맨드 라인이 아닌 python api를 이용해 파이썬으로 구현합니다.

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11s.pt')  # pretrained YOLO 모델 로드
model.train(data='PigDataset/pig.yaml', epochs=100, imgsz=640, patience=20)  # train
# model('/content/PigDataset/val/images/001_W5741__2021-10-08_14_45_07_frame0005.bmp')  # predict (image)
# model('./PigDataset/images/val')  # predict (directory)


In [ ]:
# 모델 평가(AP 점수)
metrics = model.val(plots=True)  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

In [ ]:
print(metrics.confusion_matrix.to_df())

예측 결과 및 GT 비교
* Python으로 구현할 때의 장점

In [ ]:
def read_yolo_labels(label_path, img_w, img_h):
    """
    YOLO 형식 (cls x_center_norm y_center_norm w_norm h_norm) 라인을 읽어
    이미지 픽셀 좌표 (x1,y1,x2,y2)와 클래스 인덱스 리스트로 반환
    """
    boxes, cls_idxs = [], []
    with open(label_path, 'r') as f:
      if f is None:
        return [], []
      else:
        for line in f:
          parts = line.strip().split()

          if len(parts) != 5:
              continue

          cls, xc, yc, w, h = map(float, parts)

          # normalized → 절대 좌표
          xc *= img_w
          yc *= img_h
          w  *= img_w
          h  *= img_h

          x1 = xc - w/2
          y1 = yc - h/2
          x2 = xc + w/2
          y2 = yc + h/2

          boxes.append((x1, y1, x2, y2))

          cls_idxs.append(int(cls))

    return boxes, cls_idxs

In [ ]:
from matplotlib import pyplot as plt
import glob
import cv2
import os

img_dir = './PigDataset/val/images'
img_list = glob.glob(img_dir + '/*.bmp')
img_list.sort()

label_dir = './PigDataset/val/labels'

for img_path in img_list:
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_h, img_w, _ = img.shape

  label_path = os.path.join(label_dir, os.path.basename(img_path).split('.')[0] + '.txt')
  gt_boxes, _  = read_yolo_labels(label_path, img_w, img_h)

  if gt_boxes == []:
    continue

  else:
    for (x1, y1, x2, y2) in gt_boxes:
      cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    preds = model(img, verbose=False) # 이 모델은 바로 직전 python api로 학습한 모델입니다.

    for pred in preds:
      for box, conf in zip(pred.boxes.xyxy, pred.boxes.conf):
        conf = conf.cpu().numpy()
        x1, y1, x2, y2 = box
        cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), (255,0,0), 2)
        cv2.putText(img, f'{conf:.3f}', (int(x2), int(y1)-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)

  plt.figure(figsize=(10,10))
  plt.imshow(img)
  plt.axis('off')
  plt.show()
